# SHACL tutorial

This tutorial is based on the youtube webinar of Holger Knublauch  [SHACL Tutorial](https://www.youtube.com/watch?v=ccs-KhnWR1U)
    
For SKOS, see also
 [SKOS Tutorial](https://www.youtube.com/watch?v=3Q4v6vzj3Qk) by [DataFlair](https://www.youtube.com/channel/UCs6nmQZQ6pO1qG0e2fV6G7g).

and
 [SKOS Primer](https://www.w3.org/TR/skos-primer/) and the [SKOS Reference](https://www.w3.org/TR/skos-reference/).

In [24]:
import rdflib

import matplotlib
import matplotlib.pyplot as plt

from pyshacl import Validator
from rdflib import Graph
# inline matplotlib plotting in jupyter notebook

%matplotlib inline

In [25]:
# create a simple SKOS RDF graph (turtle format)

simple_skos_ttl_graph = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix ex: <http://example.org/> .
@prefix shdemo: <http://w3id.org/oso/shacl-skos-demo-ontology/> .

<http://w3id.org/oso/shacl-skos-demo-ontology> rdf:type owl:Ontology .


# City --------------------------------------------------------------

shdemo:City
    a owl:Class ;
    a sh:NodeShape ;
    rdfs:label "City"@en ;
    rdfs:subClassOf skos:Concept;
    sh:property shdemo:City-Population;
    .
# property shapes for City
shdemo:City-Population
    a sh:PropertyShape ;
    sh:path shdemo:population ;
    sh:datatype xsd:integer ;
    sh:minCount 1 ;
    sh:name "population"@en ;
    .

    
## Country --------------------------------------------------------------
shdemo:Country
    a owl:Class ;
    a sh:NodeShape ;
    rdfs:label "Country"@en ;
    rdfs:subClassOf skos:Concept;
    sh:property shdemo:Country-isoCode;
    sh:property shdemo:Country-capital;
    .

# property shapes for Country
# ISO Code
shdemo:Country-isoCode
    a sh:PropertyShape ;
    sh:path shdemo:isoCode ;
    sh:datatype xsd:string ;
    sh:minLength 2 ;
    sh:maxLength 2 ;
    sh:regex "[A-Z]{2}" ;
    sh:name "ISO Code"@en ;
    .    

 # Capital
shdemo:Country-capital
    a sh:PropertyShape ;
    sh:path shdemo:capital ;
    sh:class shdemo:City ;
    sh:datatype xsd:string ;
    sh:maxCount 1 ;
    sh:name "Capital"@en ;
    .   
# Capital of Country (inverse)

shdemo:City-capitalOf
    a sh:PropertyShape ;
    sh:path shdemo:capital ;
    sh:inversePath shdemo:Country-capital ;
    sh:name "capitalOf"@en ;
    .

"""


In [ ]:
shape_graph = rdflib.Graph()
shape_graph.parse(data=simple_skos_ttl_graph, format="turtle")



In [27]:
# create a Dataset that fullfills the SHACL constraints

valid_dataset = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

@prefix ex: <http://example.org/> .
@prefix shdemo: <http://w3id.org/oso/shacl-skos-demo-ontology/> .

ex:Germany a shdemo:Country ;
    shdemo:isoCode "DE" ;
    shdemo:capital "Berlin" ;
    shdemo:population 3769495 ;
    .

ex:Berlin a shdemo:City ;
    shdemo:population 3769495 ;
    .

ex:France a shdemo:Country ;
    shdemo:isoCode "FR" ;
    shdemo:capital "Paris" ;
    shdemo:population 2140526 ;
    .

ex:Paris a shdemo:City ;
    shdemo:population 2140526 ;
    .

"""

In [51]:
# create a Dataset that violates the SHACL constraints

invalid_dataset = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix ex: <http://example.org/> .
@prefix shdemo: <http://w3id.org/oso/shacl-skos-demo-ontology/> .

ex:Germany a shdemo:Country ;
    shdemo:isoCode "DE3" ;
    shdemo:capital "Berlin" ;
    shdemo:population 0 ;
    .

ex:Berlin a shdemo:City ;
    shdemo:population 3769495 ;
    .

ex:France a shdemo:Country ;
    shdemo:isoCode "fr" ;
    shdemo:capital "Paris" ;
    shdemo:population 2140526 ;
    .

ex:Paris a shdemo:City ;
    shdemo:population 2140526 ;
    .

ex:Italy a shdemo:Country ;
    shdemo:isoCode "ITA" ;
    shdemo:capital "Rome" ;
    shdemo:capital "Paris" ;
    shdemo:population 2872800 ;
    .

ex:Rome a shdemo:City ;
    shdemo:population 0 ;
    .

"""


In [ ]:
valid_graph = rdflib.Graph()
valid_graph.parse(data=valid_dataset, format="turtle")


In [ ]:
# validate the valid dataset with pyshacl

v = Validator(valid_graph, shacl_graph=shape_graph,  allow_warnings=True ) # inference='rdfs', abort_on_error=False,

conforms, results_graph, results_text = v.run()
conforms, results_graph, results_text

In [ ]:
invalid_graph = rdflib.Graph()
invalid_graph.parse(data=invalid_dataset, format="turtle")

In [ ]:
# validate the invalid dataset with pyshacl

v = Validator(invalid_graph, shacl_graph=shape_graph, inference='rdfs', abort_on_error=False  )

conforms, results_graph, results_text = v.run()

conforms, results_graph, print(results_text)